In [2]:
import tensorflow as tf
import gc
tf.keras.backend.clear_session()
gc.collect()


105

In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext, ttk
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing import image
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
import os
from datetime import datetime
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Create a directory to save images
IMAGE_LOG_DIR = "image_logs"
if not os.path.exists(IMAGE_LOG_DIR):
    os.makedirs(IMAGE_LOG_DIR)

# ✅ Load the trained model with custom focal loss
def focal_loss(alpha=0.25, gamma=2.0):
    def focal_loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        ce_loss = -y_true * tf.math.log(y_pred)
        focal_weight = tf.pow(1.0 - y_pred, gamma) * y_true
        return tf.reduce_mean(alpha * focal_weight * ce_loss)
    return focal_loss_fn

# Define the path to the trained model
model_path = r"C:\Users\rushi\OneDrive\Desktop\Data\covid_pneumonia_detection_best_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects={'loss': focal_loss(alpha=0.25, gamma=2.0)})

# ✅ Function to Validate if the Image is an X-ray
def is_xray_image(img_path):
    """
    Validate if the uploaded image is likely an X-ray based on simple heuristics.
    Handles .jpg files that are effectively grayscale (R=G=B).

    Args:
        img_path (str): Path to the image file.

    Returns:
        bool: True if the image is likely an X-ray, False otherwise.
    """
    try:
        # Load the image using PIL
        img = Image.open(img_path)
        
        # Convert to numpy array for analysis
        img_array = np.array(img)
        
        # Check if the image is grayscale or effectively grayscale
        if len(img_array.shape) == 3 and img_array.shape[2] == 3:  # RGB image
            # Check if R, G, B channels are identical (or nearly identical)
            r, g, b = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
            if not np.allclose(r, g, atol=5) or not np.allclose(g, b, atol=5):
                # Channels are not identical, so it's a color image, not an X-ray
                logging.debug("Image is RGB with different channels, not an X-ray.")
                return False
            # If channels are identical, convert to grayscale for further checks
            img_array = r  # Use one channel since they are the same
            logging.debug("Image is effectively grayscale (R=G=B).")
        elif len(img_array.shape) == 2:  # Grayscale image
            logging.debug("Image is already grayscale.")
        else:
            logging.debug("Image has unexpected dimensions.")
            return False

        # Check pixel intensity range (X-rays typically have high contrast)
        # Normalize the image to [0, 1] for consistency
        img_array = img_array.astype(np.float32) / 255.0
        mean_intensity = np.mean(img_array)
        std_intensity = np.std(img_array)

        # Heuristic: X-rays often have a mean intensity between 0.2 and 0.8 (after normalization)
        # and a standard deviation greater than 0.1 (indicating high contrast)
        if not (0.2 < mean_intensity < 0.8 and std_intensity > 0.1):
            logging.debug(f"Image does not match X-ray intensity profile: mean={mean_intensity}, std={std_intensity}")
            return False

        return True

    except Exception as e:
        logging.error(f"Error validating X-ray image: {str(e)}")
        return False

# ✅ Preprocess Image Function
def preprocess_image(img_path, target_size=(224, 224), grayscale=True, normalization="simple"):
    try:
        # Load original image to get its size
        original_img = Image.open(img_path)
        original_size = original_img.size  # (width, height)
        logging.debug(f"Original image size: {original_size}")

        # Save the original image
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        original_save_path = os.path.join(IMAGE_LOG_DIR, f"original_{timestamp}.png")
        original_img.save(original_save_path)
        logging.debug(f"Saved original image to: {original_save_path}")

        # Load and preprocess the image
        if grayscale:
            img = image.load_img(img_path, color_mode='grayscale', target_size=target_size)
            img_array = image.img_to_array(img)  # Shape: (224, 224, 1)
            logging.debug(f"Image shape after loading (grayscale): {img_array.shape}")
            # Replicate the grayscale channel to 3 channels for the model
            img_array_for_model = np.repeat(img_array, 3, axis=-1)  # Shape: (224, 224, 3)
            logging.debug(f"Image shape after channel replication (for model): {img_array_for_model.shape}")
        else:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)  # Shape: (224, 224, 3)
            img_array_for_model = img_array
            logging.debug(f"Image shape after loading (RGB): {img_array.shape}")

        # Normalization
        if normalization == "simple":
            img_array_for_model = img_array_for_model / 255.0  # Scale to [0, 1]
        elif normalization == "imagenet":
            img_array_for_model = img_array_for_model / 255.0
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_array_for_model = (img_array_for_model - mean) / std
        elif normalization == "custom":
            img_array_for_model = (img_array_for_model - np.mean(img_array_for_model)) / (np.std(img_array_for_model) + 1e-8)

        # Log the min and max values after normalization
        logging.debug(f"Image min/max after normalization (for model): {img_array_for_model.min()}/{img_array_for_model.max()}")

        # Save the preprocessed image as a grayscale image for display
        preprocessed_img = Image.fromarray(np.uint8(img_array.squeeze()), mode='L')  # Shape: (224, 224)
        preprocessed_size = preprocessed_img.size  # Should be (224, 224)
        logging.debug(f"Preprocessed image size (PIL): {preprocessed_size}")
        preprocessed_save_path = os.path.join(IMAGE_LOG_DIR, f"preprocessed_{timestamp}.png")
        preprocessed_img.save(preprocessed_save_path)
        logging.debug(f"Saved preprocessed image to: {preprocessed_save_path}")

        # Add batch dimension for the model
        img_array_for_model = np.expand_dims(img_array_for_model, axis=0)  # Shape: (1, 224, 224, 3)
        logging.debug(f"Final image shape for model: {img_array_for_model.shape}")

        return img, img_array_for_model, original_size, preprocessed_size, original_save_path, preprocessed_save_path
    except Exception as e:
        logging.error(f"Error in preprocess_image: {str(e)}")
        raise

# ✅ Generate Grad-CAM
def generate_grad_cam(model, img_path, class_labels, layer_name="conv2d_2"):
    logging.debug(f"Processing image: {img_path}")
    try:
        # Preprocess the image
        img, img_array, original_size, preprocessed_size, original_save_path, preprocessed_save_path = preprocess_image(
            img_path,
            target_size=(224, 224),
            grayscale=True,
            normalization="simple"
        )

        # Model prediction
        preds = model.predict(img_array)
        pred_class_idx = np.argmax(preds)
        pred_class_label = class_labels[pred_class_idx]
        confidence = np.max(preds) * 100
        confidence_all = preds[0] * 100

        # Grad-CAM computation
        grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(img_array)
            loss = predictions[:, pred_class_idx]

        grads = tape.gradient(loss, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        conv_outputs = conv_outputs[0]
        heatmap = np.mean(conv_outputs * pooled_grads, axis=-1)
        heatmap = np.maximum(heatmap, 0)
        heatmap /= (np.max(heatmap) + 1e-8)
        heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (224, 224)).numpy().squeeze()

        heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
        superimposed_img = cv2.addWeighted(cv2.cvtColor(np.array(img), cv2.COLOR_GRAY2BGR), 0.6, heatmap_colored, 0.4, 0)

        return img, heatmap_resized, superimposed_img, pred_class_label, confidence, confidence_all, original_size, preprocessed_size, original_save_path, preprocessed_save_path
    except Exception as e:
        logging.error(f"Error in generate_grad_cam: {str(e)}")
        raise

#  Explain Prediction
def explain_prediction(pred_class, heatmap, confidence_all):
    left_lung = np.mean(heatmap[:, :112])
    right_lung = np.mean(heatmap[:, 112:])
    lung_activation = max(left_lung, right_lung)

    explanation = ""
    if pred_class == "COVID19":
        explanation += " COVID-19 Detected\n"
        explanation += "- Heatmap shows bilateral lung involvement, with intense activation in lower lungs.\n"
        explanation += "- This suggests ground-glass opacities (GGO), a common COVID pattern.\n"
        explanation += "- Strong activation: Possible severe infection.\n" if lung_activation > 0.5 else "- Moderate activation: Early-stage or mild infection.\n"
        explanation += "\n Why NOT Pneumonia?\n- Pneumonia usually shows unilateral consolidation, unlike this bilateral pattern.\n"
    elif pred_class == "PNEUMONIA":
        explanation += " Pneumonia Detected\n"
        explanation += "- Model focuses on one lung region, suggesting lobar consolidation.\n"
        explanation += "- Left lung stronger.\n" if left_lung > right_lung else "- Right lung stronger.\n"
        explanation += "\n Why NOT COVID-19?\n- COVID-19 typically shows bilateral diffuse opacities, unlike this localized pattern.\n"
    elif pred_class == "NORMAL":
        explanation += "No Disease Detected\n"
        explanation += "- Heatmap shows no strong lung focus, indicating no abnormalities.\n"

    simple_explanation = "\n What This Means for You\n"
    if pred_class == "COVID19":
        simple_explanation += "- The model thinks you might have COVID-19, a viral infection affecting both lungs. The highlighted areas show where the \n"
        simple_explanation += "infection might be. You may need to isolate and consult a doctor for a test or treatment.\n"
    elif pred_class == "PNEUMONIA":
        simple_explanation += "- The model suggests you might have pneumonia, a lung infection often caused by bacteria. It’s affecting one lung more than the other. You should see a doctor for antibiotics or further tests.\n"
    elif pred_class == "NORMAL":
        simple_explanation += "- Good news! The model doesn’t see any signs of disease in your lungs. Your X-ray looks normal, but always check with a doctor if you have symptoms.\n"

    confidence_breakdown = "\n Confidence Scores\n"
    confidence_breakdown += f"- COVID-19: {confidence_all[0]:.2f}%\n"
    confidence_breakdown += f"- NORMAL: {confidence_all[1]:.2f}%\n"
    confidence_breakdown += f"- PNEUMONIA: {confidence_all[2]:.2f}%\n"

    return explanation + simple_explanation + confidence_breakdown

# Generate PDF Report
def generate_pdf_report(img_path, heatmap, superimposed_img, pred_class, confidence, confidence_all, explanation, patient_info):
    pdf_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
    if not pdf_path:
        return

    try:
        logging.debug(f"Generating PDF at: {pdf_path}")
        c = canvas.Canvas(pdf_path, pagesize=letter)
        width, height = letter

        # Header with Patient Info
        c.setFillColor(colors.darkblue)
        c.setFont("Helvetica-Bold", 16)
        c.drawString(50, height - 50, "COVID Pneumonia Detection Report")
        c.setFont("Helvetica", 10)
        c.setFillColor(colors.black)
        c.drawString(50, height - 70, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        c.drawString(50, height - 90, f"Patient ID: {patient_info['id'] if patient_info['id'] else 'Not Provided'}")
        c.drawString(50, height - 110, f"Name: {patient_info['name'] if patient_info['name'] else 'Not Provided'}")
        c.drawString(50, height - 130, f"Age: {patient_info['age'] if patient_info['age'] else 'Not Provided'}")
        c.drawString(50, height - 150, f"Gender: {patient_info['gender'] if patient_info['gender'] else 'Not Provided'}")

        # Images
        c.setFont("Helvetica-Bold", 12)
        logging.debug("Saving original image")
        c.drawString(50, height - 180, "Original Image")
        img = Image.open(img_path).resize((150, 150))
        img.save("temp_original.png", format="PNG")
        c.drawImage(ImageReader("temp_original.png"), 50, height - 350, width=150, height=150)

        logging.debug("Saving heatmap image")
        c.drawString(250, height - 180, "Grad-CAM Heatmap")
        heatmap_img = Image.fromarray(np.uint8(255 * heatmap), mode='L').convert('RGB').resize((150, 150))
        heatmap_img.save("temp_heatmap.png", format="PNG")
        c.drawImage(ImageReader("temp_heatmap.png"), 250, height - 350, width=150, height=150)

        logging.debug("Saving superimposed image")
        c.drawString(450, height - 180, "Superimposed Image")
        superimposed_img_pil = Image.fromarray(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)).resize((150, 150))
        superimposed_img_pil.save("temp_superimposed.png", format="PNG")
        c.drawImage(ImageReader("temp_superimposed.png"), 450, height - 350, width=150, height=150)

        # Heatmap Explanation
        c.setFont("Helvetica", 10)
        c.drawString(250, height - 370, "Heatmap Color Guide: Red = High Focus, Blue = Low Focus")

        # Prediction
        c.setFont("Helvetica-Bold", 12)
        c.drawString(50, height - 400, f"Prediction: {pred_class} ({confidence:.2f}%)")

        # Explanation
        c.setFont("Helvetica-Bold", 12)
        c.drawString(50, height - 430, "Detailed Explanation:")
        text = c.beginText(50, height - 450)
        text.setFont("Helvetica", 10)
        for line in explanation.split("\n"):
            text.textLine(line.replace("**", ""))
        c.drawText(text)

        # Calculate the height of the explanation text to position the glossary
        explanation_lines = len(explanation.split("\n"))
        glossary_y_start = (height - 450) - (explanation_lines * 12)

        # Glossary
        c.setFont("Helvetica-Bold", 12)
        c.drawString(50, glossary_y_start - 20, "Glossary of Terms:")
        text = c.beginText(50, glossary_y_start - 40)
        text.setFont("Helvetica", 10)
        text.textLine("Ground-Glass Opacities (GGO): Hazy areas in the lungs, often seen in COVID-19.")
        text.textLine("Lobar Consolidation: Dense, filled lung area, common in bacterial pneumonia.")
        text.textLine("Bilateral: Affecting both lungs.")
        text.textLine("Unilateral: Affecting one lung.")
        c.drawText(text)

        # Footer
        c.setFont("Helvetica", 8)
        c.setFillColor(colors.gray)
        c.drawString(50, 50, "Disclaimer: This is an AI-based prediction. Consult a doctor for a medical diagnosis.")
        c.drawString(50, 40, "Contact: support@healthai.com")

        c.showPage()
        c.save()

        # Clean up temporary files
        for temp_file in ["temp_original.png", "temp_heatmap.png", "temp_superimposed.png"]:
            if os.path.exists(temp_file):
                os.remove(temp_file)

        messagebox.showinfo("Success", f"Report saved as {pdf_path}")

    except Exception as e:
        logging.error(f"Error generating PDF: {str(e)}")
        messagebox.showerror("Error", f"Failed to generate PDF: {str(e)}")

# ✅ GUI Setup
def predict_and_display():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.jpeg")])
    if not file_path:
        return

    # Validate if the image is an X-ray
    if not is_xray_image(file_path):
        messagebox.showerror("Error", "The uploaded image does not appear to be an X-ray. Please upload a valid chest X-ray image.")
        return

    # Collect patient info
    patient_info = {
        'id': patient_id_entry.get(),
        'name': name_entry.get(),
        'age': age_entry.get(),
        'gender': gender_var.get()
    }
    progress["value"] = 0
    root.update()

    try:
        progress["value"] = 30
        root.update()

        # Generate Grad-CAM and prediction
        img, heatmap, superimposed_img, pred_class, confidence, confidence_all, original_size, preprocessed_size, original_save_path, preprocessed_save_path = generate_grad_cam(
            model, file_path, ["COVID19", "NORMAL", "PNEUMONIA"]
        )

        progress["value"] = 60
        root.update()

        # Display images with error handling
        try:
            # Original Image
            img = img.resize((200, 200))
            img_tk = ImageTk.PhotoImage(img)
            original_label.config(image=img_tk)
            original_label.image = img_tk  # Keep a reference

            # Heatmap Image
            heatmap_img = Image.fromarray(np.uint8(255 * heatmap), mode='L').convert('RGB')
            heatmap_img = heatmap_img.resize((200, 200))
            heatmap_tk = ImageTk.PhotoImage(heatmap_img)
            heatmap_label.config(image=heatmap_tk)
            heatmap_label.image = heatmap_tk  # Keep a reference

            # Superimposed Image
            superimposed_img_pil = Image.fromarray(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
            superimposed_img_pil = superimposed_img_pil.resize((200, 200))
            superimposed_tk = ImageTk.PhotoImage(superimposed_img_pil)
            superimposed_label.config(image=superimposed_tk)
            superimposed_label.image = superimposed_tk  # Keep a reference

            # Display original and preprocessed images side by side
            original_display_img = Image.open(original_save_path).resize((150, 150))
            original_display_tk = ImageTk.PhotoImage(original_display_img)
            original_preprocess_label.config(image=original_display_tk)
            original_preprocess_label.image = original_display_tk  # Keep a reference
            original_size_label.config(text=f"Size: {original_size[0]}x{original_size[1]}")

            preprocessed_display_img = Image.open(preprocessed_save_path).resize((150, 150))
            preprocessed_display_tk = ImageTk.PhotoImage(preprocessed_display_img)
            preprocessed_label.config(image=preprocessed_display_tk)
            preprocessed_label.image = preprocessed_display_tk  # Keep a reference
            preprocessed_size_label.config(text=f"Size: {preprocessed_size[0]}x{preprocessed_size[1]}")

        except Exception as e:
            logging.error(f"Error displaying images: {str(e)}")
            messagebox.showerror("Error", f"Failed to display images: {str(e)}")
            return

        progress["value"] = 80
        root.update()

        # Update prediction and explanation
        result_label.config(text=f"Prediction: {pred_class} ({confidence:.2f}%)")
        explanation = explain_prediction(pred_class, heatmap, confidence_all)
        explanation_text.delete(1.0, tk.END)
        explanation_text.insert(tk.END, explanation)

        # Confidence bar
        confidence_canvas.delete("all")
        confidence_canvas.create_rectangle(0, 0, confidence * 2, 20, fill="green")
        confidence_canvas.create_text(100, 10, text=f"{confidence:.2f}%", fill="white")

        progress["value"] = 100
        root.update()

        # Enable download button
        download_button.config(state=tk.NORMAL)
        download_button.config(command=lambda: generate_pdf_report(file_path, heatmap, superimposed_img, pred_class, confidence, confidence_all, explanation, patient_info))

        # Update the canvas scroll region after adding content
        content_frame.update_idletasks()
        tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

    except Exception as e:
        logging.error(f"Error in prediction: {str(e)}")
        messagebox.showerror("Error", f"An error occurred: {str(e)}")
        progress["value"] = 0

def clear_display():
    original_label.config(image='')
    heatmap_label.config(image='')
    superimposed_label.config(image='')
    original_preprocess_label.config(image='')
    preprocessed_label.config(image='')
    original_size_label.config(text="Size: N/A")
    preprocessed_size_label.config(text="Size: N/A")
    result_label.config(text="Prediction: None")
    explanation_text.delete(1.0, tk.END)
    confidence_canvas.delete("all")
    download_button.config(state=tk.DISABLED)
    progress["value"] = 0
    patient_id_entry.delete(0, tk.END)
    name_entry.delete(0, tk.END)
    age_entry.delete(0, tk.END)
    gender_var.set("")

    # Update the canvas scroll region after clearing content
    content_frame.update_idletasks()
    tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

def toggle_theme():
    if theme_var.get() == "Light":
        root.config(bg="#E6F0FA")
        content_frame.config(bg="#E6F0FA")
        patient_frame.config(bg="#E6F0FA")
        button_frame.config(bg="#E6F0FA")
        frame.config(bg="white")
        preprocess_frame.config(bg="white")
        explanation_text.config(bg="white", fg="black")
    else:
        root.config(bg="#2E2E2E")
        content_frame.config(bg="#2E2E2E")
        patient_frame.config(bg="#2E2E2E")
        button_frame.config(bg="#2E2E2E")
        frame.config(bg="#3C3C3C")
        preprocess_frame.config(bg="#3C3C3C")
        explanation_text.config(bg="#3C3C3C", fg="white")

# Initialize GUI
root = tk.Tk()
root.title("COVID Pneumonia Detector")
root.geometry("900x600")  # Reduced height since we'll add a scrollbar

# Create a canvas and scrollbar (renamed to tk_canvas to avoid conflict)
tk_canvas = tk.Canvas(root, bg="#E6F0FA")
scrollbar = ttk.Scrollbar(root, orient=tk.VERTICAL, command=tk_canvas.yview)
tk_canvas.configure(yscrollcommand=scrollbar.set)

# Pack the canvas and scrollbar
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
tk_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# Create a frame inside the canvas to hold all content
content_frame = tk.Frame(tk_canvas, bg="#E6F0FA")
tk_canvas.create_window((0, 0), window=content_frame, anchor="nw")

# Patient Info Input
patient_frame = tk.Frame(content_frame, bg="#E6F0FA")
patient_frame.pack(pady=5)

tk.Label(patient_frame, text="Patient ID:", font=("Arial", 12), bg="#E6F0FA").pack(side=tk.LEFT, padx=5)
patient_id_entry = tk.Entry(patient_frame, font=("Arial", 12))
patient_id_entry.pack(side=tk.LEFT, padx=5)

tk.Label(patient_frame, text="Name:", font=("Arial", 12), bg="#E6F0FA").pack(side=tk.LEFT, padx=5)
name_entry = tk.Entry(patient_frame, font=("Arial", 12))
name_entry.pack(side=tk.LEFT, padx=5)

tk.Label(patient_frame, text="Age:", font=("Arial", 12), bg="#E6F0FA").pack(side=tk.LEFT, padx=5)
age_entry = tk.Entry(patient_frame, font=("Arial", 12), width=5)
age_entry.pack(side=tk.LEFT, padx=5)

tk.Label(patient_frame, text="Gender:", font=("Arial", 12), bg="#E6F0FA").pack(side=tk.LEFT, padx=5)
gender_var = tk.StringVar()
gender_menu = ttk.Combobox(patient_frame, textvariable=gender_var, values=["", "Male", "Female", "Other"], state="readonly", width=10)
gender_menu.pack(side=tk.LEFT, padx=5)

# Buttons Frame
button_frame = tk.Frame(content_frame, bg="#E6F0FA")
button_frame.pack(pady=10)

style = ttk.Style()
style.configure("TButton", font=("Arial", 12), padding=10)
style.map("TButton", background=[("active", "#A3BFFA")])

ttk.Button(button_frame, text="Upload X-ray Image", command=predict_and_display).pack(side=tk.LEFT, padx=5)
ttk.Button(button_frame, text="Clear", command=clear_display).pack(side=tk.LEFT, padx=5)

# Theme Toggle
theme_var = tk.StringVar(value="Light")
theme_menu = ttk.OptionMenu(button_frame, theme_var, "Light", "Light", "Dark", command=lambda _: toggle_theme())
theme_menu.pack(side=tk.LEFT, padx=5)

# Progress Bar
progress = ttk.Progressbar(content_frame, length=300, mode="determinate")
progress.pack(pady=5)

# Image display frames (Original, Heatmap, Superimposed)
frame = tk.Frame(content_frame, bg="white", bd=2, relief=tk.SUNKEN)
frame.pack(pady=10)

tk.Label(frame, text="Original Image", font=("Arial", 10, "bold"), bg="white").grid(row=0, column=0, padx=10)
original_label = tk.Label(frame, bg="white")
original_label.grid(row=1, column=0)

tk.Label(frame, text="Grad-CAM Heatmap", font=("Arial", 10, "bold"), bg="white").grid(row=0, column=1, padx=10)
heatmap_label = tk.Label(frame, bg="white")
heatmap_label.grid(row=1, column=1)

tk.Label(frame, text="Superimposed Image", font=("Arial", 10, "bold"), bg="white").grid(row=0, column=2, padx=10)
superimposed_label = tk.Label(frame, bg="white")
superimposed_label.grid(row=1, column=2)

# Preprocessing display frame (Original vs Preprocessed)
preprocess_frame = tk.Frame(content_frame, bg="white", bd=2, relief=tk.SUNKEN)
preprocess_frame.pack(pady=10)

tk.Label(preprocess_frame, text="Original", font=("Arial", 10, "bold"), bg="white").grid(row=0, column=0, padx=20)
original_preprocess_label = tk.Label(preprocess_frame, bg="white")
original_preprocess_label.grid(row=1, column=0)
original_size_label = tk.Label(preprocess_frame, text="Size: N/A", font=("Arial", 8), bg="white")
original_size_label.grid(row=2, column=0)

tk.Label(preprocess_frame, text="Preprocessed", font=("Arial", 10, "bold"), bg="white").grid(row=0, column=1, padx=20)
preprocessed_label = tk.Label(preprocess_frame, bg="white")
preprocessed_label.grid(row=1, column=1)
preprocessed_size_label = tk.Label(preprocess_frame, text="Size: N/A", font=("Arial", 8), bg="white")
preprocessed_size_label.grid(row=2, column=1)

# Prediction and Confidence
result_label = tk.Label(content_frame, text="Prediction: None", font=("Arial", 12, "bold"), bg="#E6F0FA")
result_label.pack(pady=5)

confidence_frame = tk.Frame(content_frame, bg="#E6F0FA")
confidence_frame.pack(pady=5)
tk.Label(confidence_frame, text="Confidence:", font=("Arial", 10), bg="#E6F0FA").pack(side=tk.LEFT)
confidence_canvas = tk.Canvas(confidence_frame, width=200, height=20, bg="white")
confidence_canvas.pack(side=tk.LEFT)

# Explanation
explanation_text = scrolledtext.ScrolledText(content_frame, width=80, height=10, wrap=tk.WORD, font=("Arial", 10), bg="white")
explanation_text.pack(pady=10)

# Download Report Button
download_button = ttk.Button(content_frame, text="Download Report", state=tk.DISABLED)
download_button.pack(pady=10)

# Update the canvas scroll region after initial setup
content_frame.update_idletasks()
tk_canvas.config(scrollregion=tk_canvas.bbox("all"))

# Bind mouse wheel scrolling to the canvas
def on_mouse_wheel(event):
    tk_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

tk_canvas.bind_all("<MouseWheel>", on_mouse_wheel)

# Start GUI
root.mainloop()

2025-04-10 00:10:22,674 - DEBUG - Creating converter from 3 to 5
2025-04-10 00:10:55,368 - DEBUG - Image is effectively grayscale (R=G=B).
2025-04-10 00:10:55,373 - DEBUG - Processing image: C:/Users/rushi/OneDrive/Desktop/Sample Images/Screenshot 2025-04-08 160608.jpg
2025-04-10 00:10:55,374 - DEBUG - Original image size: (668, 736)
2025-04-10 00:10:55,425 - DEBUG - Saved original image to: image_logs\original_20250410_001055.png
2025-04-10 00:10:55,428 - DEBUG - Image shape after loading (grayscale): (224, 224, 1)
2025-04-10 00:10:55,429 - DEBUG - Image shape after channel replication (for model): (224, 224, 3)
2025-04-10 00:10:55,430 - DEBUG - Image min/max after normalization (for model): 0.027450980618596077/1.0
2025-04-10 00:10:55,430 - DEBUG - Preprocessed image size (PIL): (224, 224)
2025-04-10 00:10:55,433 - DEBUG - Saved preprocessed image to: image_logs\preprocessed_20250410_001055.png
2025-04-10 00:10:55,434 - DEBUG - Final image shape for model: (1, 224, 224, 3)


1/1 [==============================] - 1s 761ms/step


2025-04-10 00:10:56,601 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:10:56,601 - DEBUG - STREAM b'IDAT' 41 65536
2025-04-10 00:10:56,618 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:10:56,619 - DEBUG - STREAM b'IDAT' 41 33786
2025-04-10 00:11:21,364 - DEBUG - Generating PDF at: C:/Users/rushi/Downloads/Pneumonia report.pdf
2025-04-10 00:11:21,366 - DEBUG - Saving original image
2025-04-10 00:11:21,381 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:11:21,383 - DEBUG - STREAM b'IDAT' 41 24713
2025-04-10 00:11:21,395 - DEBUG - Saving heatmap image
2025-04-10 00:11:21,407 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:11:21,407 - DEBUG - STREAM b'IDAT' 41 14144
2025-04-10 00:11:21,418 - DEBUG - Saving superimposed image
2025-04-10 00:11:21,431 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:11:21,431 - DEBUG - STREAM b'IDAT' 41 39889
2025-04-10 00:14:23,004 - DEBUG - Image is effectively grayscale (R=G=B).
2025-04-10 00:14:23,008 - DEBUG - Processing image: C:/Users/rushi/Downloads/4994014ef5c834

1/1 [==============================] - 0s 19ms/step


2025-04-10 00:14:23,171 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:14:23,172 - DEBUG - STREAM b'IDAT' 41 65536
2025-04-10 00:14:23,186 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:14:23,187 - DEBUG - STREAM b'IDAT' 41 29117
2025-04-10 00:14:54,339 - DEBUG - Image is RGB with different channels, not an X-ray.
2025-04-10 00:14:58,515 - DEBUG - Image is effectively grayscale (R=G=B).
2025-04-10 00:14:58,523 - DEBUG - Processing image: C:/Users/rushi/OneDrive/Desktop/Sample Images/test2.jpg
2025-04-10 00:14:58,524 - DEBUG - Original image size: (1316, 882)
2025-04-10 00:14:58,638 - DEBUG - Saved original image to: image_logs\original_20250410_001458.png
2025-04-10 00:14:58,645 - DEBUG - Image shape after loading (grayscale): (224, 224, 1)
2025-04-10 00:14:58,646 - DEBUG - Image shape after channel replication (for model): (224, 224, 3)
2025-04-10 00:14:58,647 - DEBUG - Image min/max after normalization (for model): 0.0/1.0
2025-04-10 00:14:58,647 - DEBUG - Preprocessed image size (PIL): 

...

2025-04-10 00:15:37,434 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:15:37,435 - DEBUG - STREAM b'IDAT' 41 65536
2025-04-10 00:15:37,467 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:15:37,468 - DEBUG - STREAM b'IDAT' 41 31031
2025-04-10 00:15:41,409 - DEBUG - Image is already grayscale.
2025-04-10 00:15:41,468 - DEBUG - Processing image: C:/Users/rushi/OneDrive/Desktop/Sample Images/test18.jpg
2025-04-10 00:15:41,469 - DEBUG - Original image size: (3050, 2539)
2025-04-10 00:15:41,717 - DEBUG - Saved original image to: image_logs\original_20250410_001541.png
2025-04-10 00:15:41,725 - DEBUG - Image shape after loading (grayscale): (224, 224, 1)
2025-04-10 00:15:41,727 - DEBUG - Image shape after channel replication (for model): (224, 224, 3)
2025-04-10 00:15:41,727 - DEBUG - Image min/max after normalization (for model): 0.03529411926865578/1.0
2025-04-10 00:15:41,728 - DEBUG - Preprocessed image size (PIL): (224, 224)
2025-04-10 00:15:41,731 - DEBUG - Saved preprocessed image to: image_

1/1 [==============================] - 0s 16ms/step


2025-04-10 00:15:41,921 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:15:41,922 - DEBUG - STREAM b'IDAT' 41 65536
2025-04-10 00:15:42,001 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:15:42,001 - DEBUG - STREAM b'IDAT' 41 28935
2025-04-10 00:15:46,377 - DEBUG - Image is effectively grayscale (R=G=B).
2025-04-10 00:15:46,384 - DEBUG - Processing image: C:/Users/rushi/OneDrive/Desktop/Sample Images/test17.jpg
2025-04-10 00:15:46,384 - DEBUG - Original image size: (1184, 688)
2025-04-10 00:15:46,453 - DEBUG - Saved original image to: image_logs\original_20250410_001546.png
2025-04-10 00:15:46,458 - DEBUG - Image shape after loading (grayscale): (224, 224, 1)
2025-04-10 00:15:46,459 - DEBUG - Image shape after channel replication (for model): (224, 224, 3)
2025-04-10 00:15:46,461 - DEBUG - Image min/max after normalization (for model): 0.0/1.0
2025-04-10 00:15:46,462 - DEBUG - Preprocessed image size (PIL): (224, 224)
2025-04-10 00:15:46,466 - DEBUG - Saved preprocessed image to: image_logs\

1/1 [==============================] - 0s 22ms/step


2025-04-10 00:15:46,603 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:15:46,604 - DEBUG - STREAM b'IDAT' 41 65536
2025-04-10 00:15:46,634 - DEBUG - STREAM b'IHDR' 16 13
2025-04-10 00:15:46,634 - DEBUG - STREAM b'IDAT' 41 27521


In [4]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the trained model
model = load_model("covid_pneumonia_detection_best_model.h5")
class_names = ['COVID19', 'NORMAL', 'PNEUMONIA']

# Function to explain why other classes got some confidence
def explain_prediction(predictions):
    main_idx = np.argmax(predictions)
    explanation = ""
    for i, score in enumerate(predictions):
        if i != main_idx and score > 0.01:
            if class_names[i] == "PNEUMONIA":
                explanation += f"🔍 {score*100:.2f}% PNEUMONIA: Minor lung opacities may resemble early pneumonia signs.\n"
            elif class_names[i] == "COVID19":
                explanation += f"🔍 {score*100:.2f}% COVID19: Detected bilateral patchy patterns typical of COVID.\n"
            elif class_names[i] == "NORMAL":
                explanation += f"🔍 {score*100:.2f}% NORMAL: Some areas may resemble normal lung structure.\n"
    return explanation

# Prediction function
def predict_xray(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)[0]

    predicted_index = np.argmax(predictions)
    predicted_class = class_names[predicted_index]
    confidence = predictions[predicted_index] * 100

    result_text = f"🩺 Prediction: {predicted_class} ({confidence:.2f}%)\n\n"
    result_text += "📊 Confidence Scores:\n"
    for i, score in enumerate(predictions):
        result_text += f"{class_names[i]}: {score * 100:.2f}%\n"

    explanation = explain_prediction(predictions)
    if explanation:
        result_text += "\n🧠 Why model saw signs of other classes:\n" + explanation
    else:
        result_text += "\n✅ Model is highly confident. No confusing features detected."

    return result_text

# GUI setup
def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if file_path:
        img = Image.open(file_path)
        img = img.resize((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        image_label.configure(image=img_tk)
        image_label.image = img_tk
        result = predict_xray(file_path)
        result_text.configure(state='normal')
        result_text.delete("1.0", tk.END)
        result_text.insert(tk.END, result)
        result_text.configure(state='disabled')

# Build GUI
root = tk.Tk()
root.title("🧠 COVID-PNEUMONIA Detection GUI")
root.geometry("700x700")
root.configure(bg="#f0f0f0")

upload_button = tk.Button(root, text="Upload X-ray Image", command=upload_image, font=("Arial", 14), bg="#2d89ef", fg="white", padx=20, pady=10)
upload_button.pack(pady=20)

image_label = tk.Label(root)
image_label.pack(pady=10)

result_text = tk.Text(root, height=20, width=80, font=("Courier", 10))
result_text.configure(state='disabled')
result_text.pack(pady=20)

root.mainloop()


1/1 [==============================] - 0s 17ms/step


2025-04-10 00:29:51,392 - DEBUG - tag: ImageWidth (256) - type: long (4) - value: b'\xa0\x0f\x00\x00'
2025-04-10 00:29:51,393 - DEBUG - tag: ImageLength (257) - type: long (4) - value: b'\xb8\x0b\x00\x00'
2025-04-10 00:29:51,393 - DEBUG - tag: Make (271) - type: string (2) Tag Location: 46 - Data Location: 158 - value: b'samsung\x00'
2025-04-10 00:29:51,394 - DEBUG - tag: Model (272) - type: string (2) Tag Location: 58 - Data Location: 166 - value: b'Galaxy S24 FE\x00'
2025-04-10 00:29:51,394 - DEBUG - tag: Orientation (274) - type: short (3) - value: b'\x06\x00'
2025-04-10 00:29:51,394 - DEBUG - tag: XResolution (282) - type: rational (5) Tag Location: 82 - Data Location: 180 - value: b'H\x00\x00\x00\x01\x00\x00\x00'
2025-04-10 00:29:51,394 - DEBUG - tag: YResolution (283) - type: rational (5) Tag Location: 94 - Data Location: 188 - value: b'H\x00\x00\x00\x01\x00\x00\x00'
2025-04-10 00:29:51,395 - DEBUG - tag: ResolutionUnit (296) - type: short (3) - value: b'\x02\x00'
2025-04-10 00:2

1/1 [==============================] - 0s 18ms/step
